##  키워드 기반

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [2]:
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import ast

In [3]:
class movie_recommendation:
    def __init__(self, **kargs):
        self.topn = kargs.get('topn', 10) # (default) optimal value so far
        self.vote_thres = kargs.get('vote_thres', 100)
        self.df = kargs.get('data', pd.read_csv('./your/data/here.csv')) # (default) movie dataframe
        self.a, self.b, self.c = kargs.get('a',0.4), kargs.get('b',0.1), kargs.get('c',0.5)
        self.verbose = kargs.get('verbose', 1)
        
        self.cvec = CountVectorizer(min_df=0, ngram_range=(1,2))
        self.w2v = word2vec.Word2Vec.load('./model/naver_plot_kmr_min10_N_only.model')
        self.scaler = MinMaxScaler()
        
        if self.verbose == 1:
            print('-'*35)
            print('# Parameters')
            print('      a, b, c        : {0}, {1}, {2}'.format(self.a, self.b, self.c))
            print('vote count threshold :', self.vote_thres)
            print('weighted_sum = keywords*{0}(a) + genre*{1}(b) + weighted vote*{2}(c)'.format(self.a, self.b, self.c))
            print('-'*35)
        
    def search_title(self, title_name):
        return self.df[self.df['title'].str.contains(title_name)].title   
    
    def genre_sim_sorted(self, title_idx):
        genre_literal = self.df['genre'].apply(lambda x: x.replace('|',' '))
        genre = self.cvec.fit_transform(genre_literal)
        genre_sim = cosine_similarity(genre,genre)
        
        return np.array([(idx,sim) for idx,sim in enumerate(genre_sim[title_idx])])
           
    def cos_sim(self, corp1, corp2):
        vec1, vec2 = [], []
        for word1, word2 in zip(corp1,corp2):
            vec1.append(self.w2v[word1])
            vec2.append(self.w2v[word2])

        vec1, vec2 = np.array(vec1).mean(axis=0), np.array(vec2).mean(axis=0)
        return np.inner(vec1,vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2))

    def similar_keywords_movies(self, title_idx):
        self.df['keywords_literal'] = self.df['keywords'].apply(lambda x: ' '.join(ast.literal_eval(x)))
        keywords_sims = []
        for idx in self.df.index:
            keywords_src = ast.literal_eval(self.df.loc[title_idx,'keywords'])
            keywords_tgt = ast.literal_eval(self.df.loc[idx,'keywords'])
            keywords_sims.append((idx,self.cos_sim(keywords_src, keywords_tgt)))

        df_with_ksim = self.df.copy()
        df_with_ksim['keywords_sim'] = np.array(keywords_sims)[:,1]
        df_with_ksim = df_with_ksim[df_with_ksim['vote_count'] > self.vote_thres]
        
        return df_with_ksim.sort_values('keywords_sim',ascending=False)[1:]

    def result_by_weights(self, dataf):
        dataf['weighted_sum'] = dataf['keywords_sim_scaled']*self.a + dataf['wvote_scaled']*self.b + dataf['genre_scaled']*self.c
        
        return dataf.sort_values('weighted_sum', ascending=False)
    
    def getMovies(self, title):
        # no title result
        try: title_idx = self.df[self.df['title']== title].index.values[0]
        except:
            raise ValueError('There is no such title name. Search with "search_title" function')
        
        # get movies
        result = self.similar_keywords_movies(title_idx)

        # IMDB's weighted_vote
        def weighted_vote_average(record):
            v, r = record['vote_count'], record['rating']
            return (v/(v+m))*r + (m/(m+v))*c
        c = result['rating'].mean()
        m = result['vote_count'].quantile(.6)
        result['weighted_vote'] = result.apply(weighted_vote_average,axis=1)
        
        # merge with genre
        genre_sim = self.genre_sim_sorted(title_idx)
        result_with_genre = pd.merge(result, pd.Series(genre_sim[:,1], name='genre_sim'), left_on=result.index, right_on=genre_sim[:,0],)
             
        # minmax scale
        result_with_genre['keywords_sim_scaled'] = MinMaxScaler().fit_transform(result_with_genre['keywords_sim'].values.reshape(-1,1))
        result_with_genre['wvote_scaled'] = MinMaxScaler().fit_transform(result_with_genre['weighted_vote'].values.reshape(-1,1))
        result_with_genre['genre_scaled'] = MinMaxScaler().fit_transform(result_with_genre['genre_sim'].values.reshape(-1,1))
        
        # (optional)remove data genre score is 0
        no_genre_score_idx = result_with_genre[result_with_genre['genre_sim'] == 0].index
        result_with_genre.drop(no_genre_score_idx, inplace=True)
        
        result_with_genre = self.result_by_weights(result_with_genre)
        return result_with_genre.head(self.topn)

In [4]:
recom = movie_recommendation()

-----------------------------------
# Parameters
      a, b, c        : 0.4, 0.1, 0.5
vote count threshold : 100
weighted_sum = keywords*0.4(a) + genre*0.1(b) + weighted vote*0.5(c)
-----------------------------------


In [5]:
recom.search_title('엽문')

3024    엽문3: 최후의 대결
6884           엽문 3
7281           엽문 2
7613             엽문
Name: title, dtype: object

In [6]:
result = recom.getMovies(title='라라랜드')

In [7]:
result[['weighted_sum','title', 'keywords_literal', 'keywords_sim_scaled', 'genre_scaled', 'wvote_scaled']][:10]

,weighted_sum,title,keywords_literal,keywords_sim_scaled,genre_scaled,wvote_scaled
25,0.848046,위대한 쇼맨,재미 남자 세계 매료 합류 할리우드 최고 배우 제작진 팀,0.905742,0.774597,0.984509
1270,0.824322,렌트,건물 열정 레인 베니 미미 철거 마크 공연 로저 사랑,0.616036,1.000000,0.779070
157,0.800309,물랑 루즈,뮤지컬 가수 신분 성공 최고 상승 파리 지상 영국 세계,0.808542,0.774597,0.895937
260,0.783259,8 마일,마일 힙합 클럽 희망 분노 시작 디트로이트 배틀 최고,0.771840,0.774597,0.872245
366,0.756924,갓 헬프 더 걸,배경 시기 우정 이브 친구 여름 기억 소녀 스코틀랜드 필름,0.740937,0.774597,0.732504
362,0.752483,퍼햅스 러브,영화 사랑 뮤지컬 영화 남녀 감독 연기 시나리오 촬영 삼각관계 서커스 베이징,0.741938,0.774597,0.684098
514,0.749826,파가니니: 악마의 바이올리니스트,스캔들 왓슨 런던 권력 방탕 마음 바니 샬롯 단독 콘서트,0.713898,0.774597,0.769682
791,0.740663,드림걸즈,팀 야심 스타일 대신 리더 성공 매니저 절호 투입 기회,0.670358,0.774597,0.852213
738,0.736125,나인,세기 안식처 아내 욕구 작품 시작 치명 정부 최고 성공,0.678901,0.800000,0.645647
62,0.732819,오페라의 유령,오페라 하우스 파리 마음 협박 천상 목소리 매니저 지하 단원,0.866591,0.600000,0.861827


In [7]:
result[['weighted_sum','title', 'keywords_literal', 'keywords_sim_scaled', 'genre_scaled', 'wvote_scaled']][:10]

,weighted_sum,title,keywords_literal,keywords_sim_scaled,genre_scaled,wvote_scaled
8,0.867689,레이싱: 더 라이벌,대회 자동차 상금 친구 일자리 시내 레이싱 경기장 스턴 학교,0.745860,1.000000,0.693450
39,0.806927,록키 발보아,록키 챔피언 헤비급 경기 성공 트레이너 가상 복싱 경기 얘기,0.539148,1.000000,0.912681
40,0.806849,러시 : 더 라이벌,대결 트랙 문제 라이벌 차량 설계 승부 세계 천재 레이서,0.536838,1.000000,0.921136
30,0.801873,더 파이트 클럽,챔피언 이종 격투기 세계 경기 사건 대회 승리 도움 감옥,0.581321,1.000000,0.693450
82,0.697968,뉴 스텝업: 어반댄스,크루 감정 세계 열정 아버지 챔피언 강요 꿈 연습 몰두,0.340834,1.000000,0.616342
1,0.679266,디아블로: 분노의 질주,레이스 레이싱 위험천만 마련 참여 팀 음식 자동차 속도 참가,0.810815,0.577350,0.662646
97,0.678443,분노의 레이서,웨인 불법 구로 레이스 참가 담당 경주 교도관 후회 소년,0.275010,1.000000,0.684394
25,0.675676,당갈,레슬링 아버지 재능 꿈 승승장구 승리 발휘 국제 대회 선수,0.605856,0.666667,1.000000
19,0.657824,핸즈 오브 스톤,정상 주먹 포기 자존심 목표 오브 스톤 파나마 무패 트레이너,0.630919,0.666667,0.721230
7,0.652570,그랑프리,그랑프리 사고 말 최초 우승 마지막 레이스 포기 제주도 격려,0.750492,0.577350,0.636976
